# 📝 LangChain PromptTemplate 계열 정리
* [PromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html#langchain_core.prompts.prompt.PromptTemplate)
* [ChatPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html#langchain_core.prompts.chat.ChatPromptTemplate)
* [ChatMessagePromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatMessagePromptTemplate.html#langchain_core.prompts.chat.ChatMessagePromptTemplate)
* [FewShotPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.few_shot.FewShotPromptTemplate.html#langchain_core.prompts.few_shot.FewShotPromptTemplate)
* PartialPrompt

## 1. PromptTemplate
- 정의: 가장 기본적인 프롬프트 템플릿 클래스
- 역할: 문자열 기반 프롬프트를 정의하고, {변수} 자리에 동적으로 값을 채워 넣음
- 특징:
    - 단일 문장/문단 프롬프트에 적합
    - 변수 치환으로 재사용성 ↑
    - LLM에 전달할 입력을 일관된 형식으로 유지 가능
    - 사용 예시 상황: "너는 친절한 카페 직원이야. {menu}에 대해 설명해줘."

## 2. ChatPromptTemplate
- 정의: 대화형 모델(Chat Model) 전용 프롬프트 템플릿
- 역할: 여러 메시지(system, human, ai 등)를 순서대로 정의하고, 이를 하나의 대화 컨텍스트로 구성
- 특징:
    - 역할(role) 기반 메시지 관리 (system, human, ai, tool)
    - MessagesPlaceholder 를 통해 동적 대화 삽입 가능
    - ChatGPT 같은 대화형 모델에 최적화
    - 사용 예시 상황: 시스템 메시지로 역할 정의 + 사용자 입력 + 이전 대화 맥락을 함께 전달

## 3. ChatMessagePromptTemplate
- 정의: 특정 역할(role) 을 가진 메시지 하나를 템플릿화한 클래스
- 역할: system, human, ai 같은 개별 메시지를 템플릿으로 정의
- 특징:
    - ChatPromptTemplate 내부에서 메시지 단위로 활용
    - 메시지마다 변수를 치환할 수 있음
    - 사용 예시 상황: "human: {question}", "system: 너는 수학 선생님이야."

## 4. FewShotPromptTemplate
- 정의: Few-shot 학습을 위한 프롬프트 템플릿
- 역할: 여러 개의 예시(example)를 자동으로 삽입해 LLM이 패턴을 학습하도록 유도
- 특징:
    - 예시 리스트를 템플릿에 포함시켜 모델이 참고하도록 함
    - 예시 포맷을 자동으로 반복 적용 가능
    - Zero-shot → Few-shot으로 성능 향상 가능
    - 사용 예시 상황: 질의응답 예시를 여러 개 넣고, 마지막에 새로운 질문을 던져 답변 유도

## 5. PartialPrompt
- 정의: 프롬프트 템플릿의 일부 변수를 미리 채워둔 상태로 만드는 기능
- 역할: 자주 반복되는 변수 값을 고정해두고, 나머지만 실행 시점에 채움
- 특징:
    - 프롬프트 재사용성 극대화
    - 공통 맥락(예: 역할, 도메인)을 미리 세팅 가능
    - 사용 예시 상황: "너는 {role} 전문가야." 에서 role="카페 직원" 을 미리 채워두고, 질문만 매번 바꿔서 사용

### 🚀 정리
- PromptTemplate → 문자열 기반 기본 템플릿
- ChatPromptTemplate → 대화형 모델용, 여러 메시지 관리
- ChatMessagePromptTemplate → 개별 메시지 단위 템플릿
- FewShotPromptTemplate → 예시(Few-shot) 포함 프롬프트
- PartialPrompt → 일부 변수를 미리 채워둔 템플릿


In [2]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_y


# PromptTemplate

##### 1) PromptTemplate 의 from_template() 함수 사용

In [3]:

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from pprint import pprint

template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해줘."

prompt_template = PromptTemplate.from_template(template_text)

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    #model="openai/gpt-oss-120b",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

chain = prompt_template | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3})
pprint(response)


('ChatGPT는 인터넷의 방대한 텍스트에서 다음 단어를 맞히는 훈련을 통해 언어 패턴을 학습한다.  \n'
 '이 과정에서 트랜스포머라는 구조가 문장의 맥락을 기억하며 확률 기반으로 가장 적절한 단어를 선택한다.  \n'
 '사람의 피드백을 추가로 반영해 원하지 않는 답변은 줄이고 유용한 답변은 늘리는 방식으로 성능을 끌어올린다.')


##### 2) PromptTemplate 결합하기
* 동일한 Prompt 패턴을 사용하지만 여러 개의 질문을 작성해서 LLM을 실행할 수도 있음

In [4]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해줘."
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
prompt_template.format(model_name="Deepseek", count=3)

combined_prompt = (
    prompt_template
        + PromptTemplate.from_template("\n\n그리고 {model_name} 모델의 장점을 요약해줘."
        + "{model_name} 모델과 비슷한 AI 모델은 어떤 것이 있는지에 대해서는 {language2}로 설명해줘")
)

combined_prompt.format(model_name="Deepseek", count=3, language2="영어")
# print(combined_prompt)

chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name": "Deepseek", "count": 3, "language2": "영어"})

pprint(response)


('**학습 원리 (한국어 3문장)**  \n'
 'DeepSeek은 방대한 텍스트 데이터를 바탕으로 다음 토큰을 예측하는 사전 학습(pre-training) 후, 지시 '
 '따르기(instruction tuning)와 강화학습(RLHF)으로 정렬을 개선한다.  \n'
 '전문 영역(코드·수학)에 특화된 추가 학습을 통해 일반 대화뿐 아니라 복잡한 추론 능력도 강화했다.  \n'
 '모델 가중치를 공개함으로써 연구자가 직접 미세 조정·확장할 수 있도록 했다.\n'
 '\n'
 '**장점 요약**  \n'
 '오픈소스·고성능·전문 분야 최적화, 한·영·중 다국어 능력, 상대적으로 저렴한 추론 비용, 연구자가 자유롭게 수정·재배포 가능.\n'
 '\n'
 '**Similar models (English)**  \n'
 'Models comparable to DeepSeek include Meta’s Llama-2/3, Mistral-7B, '
 'Alibaba’s Qwen series, and TII’s Falcon, all of which are open-weight LLMs '
 'that balance strong reasoning with commercial-friendly licenses.')


#### PromptTemplate 의 파라미터를 배열 형태로 하여 여러개 사용하는 경우

In [5]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 3},
    {"model_name": "Gemini", "count": 4},
    {"model_name": "claude", "count": 4},
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions]
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
print("\n\n")
for prompt in formatted_prompts:
    print(type(prompt), prompt)
    response = llm.invoke(prompt)
    pprint(response.content)

['GPT-4 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.', 'Gemini 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.', 'claude 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.']



<class 'str'> GPT-4 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.
('GPT-4는 수많은 텍스트 데이터를 바탕으로 다음에 올 토큰을 예측하는 방식으로 학습됩니다.  \n'
 '인간의 피드백을 통해 보상 모델을 만들어 강화학습으로 정답倾向을 조정합니다.  \n'
 '결과적으로 통계적 패턴을 학습해 실제 이해는 없지만 자연스러운 대화를 생성합니다.')
<class 'str'> Gemini 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.
('Gemini는 텍스트·이미지·오디오 등 다양한 데이터를 동시에 처리하는 멀티모달 구조를 기반으로, 인코더-디코더 트랜스포머를 확장해 '
 '학습합니다.  \n'
 '대규모 텍스트와 이미지 쌍을 먼저 학습한 뒤, 멀티모달 시퀀스를 하나의 시리즈로 이어 붙여 다음 토큰을 예측하는 자기 지도 방식으로 '
 '학습합니다.  \n'
 '학습이 진행되면서 멀티모달 혼합 전문가(MoE) 레이어가 활성화되어 필요한 전문가 네트워크만 선택적으로 사용해 효율을 높입니다.  \n'
 '지도 미세조정과 강화학습을 반복해 인간의 선호도에 맞춘 답변을 생성하도록 최적화됩니다.')
<class 'str'> claude 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.
('Claude는 방대한 텍스트 데이터를 학습해 언어 패턴을 익힌 뒤, 주어진 맥락에서 다음에 올 단어를 예측하는 방식으로 학습됩니다.  \n'
 '인간이 작성한 프롬프트와 응답 쌍을 비교하며 보상 모델의 피드백으로 정답에 가까운 답변을 생성하도록 미세조정됩니다.  \n'
 '강화학습과 인간 피드백(RLHF)을 반복해 유용성, 정직성, 무해성을 동시에 높이는 방향으로 최적화됩니다.  \n'
 '결국 통계적 확률 계산만으로도

# ChatPromptTemplate

* Tuple 형태의 system, user, assistant 메시지 지원
* 여러 개의 메시지를 조합하여 LLM에게 전달 가능
* 간결성과 가독성이 높고 단순한 구조

In [6]:
from langchain_core.prompts import ChatPromptTemplate
# 튜플 형태의 메시지 목록으로 프롬프트 생성 (type, content)
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "{topic}에 대해 넌 아주 자세히 알고 있기에 답변도 명확하고 자세하게 해줄 수 있어."),
        ("human", "{model_name} 모델의 학습 원리를 한글로 설명해줘")
    ]
)

messages = chat_prompt.format_messages(topic="AI", model_name="Upstage")

# 생성한 메시지 바로 주입해서 호출출
response = llm.invoke(messages)
print(response.content)

Upstage에서 배포하는 거대언어모델(이하 ‘업스테이지 모델’)은 GPT-3·GPT-4 같은 일반적인 트랜스포머 디코더 계열 모델과 큰 틀에서 동일한 ‘사전학습(Pre-training) → 미세조정(Fine-tuning) → 정렬(Alignment)’ 3단계를 거친다. 핵심은 각 단계에서 한국어·한국 업무 도메인 특화된 데이터와 기술을 집어넣어 “한국어에 특화된 업무 성능”을 끌어올리는 것이다. 아래에서 핵심 원리만 한글로 풀어 설명한다.

-------------------------------------------------
1. 사전학습(Pre-training) 단계
-------------------------------------------------
1) 학습 목표  
주어진 토큰 시퀀스의 다음 토큰을 맞추는 “자기회귀(Self-Regression)” 문제를 푼다.  
예) “오늘 날씨가 ___” → “좋다”를 예측

2) 신경망 구조  
- 트랜스포머 디코더 블록 × N층  
- Multi-Head Self-Attention + FFN + 잔차연결 + LayerNorm  
- 어텐션 마스크: 미래 토큰을 못 보게(캐주얼 마스크)  
- 위치 임베딩: 회전 위치 임베딩(RoPE)이나 ALiBi 등 최신 위치 인코딩 사용  
- 활성화 함수: SwiGLU, GELU 등 변형

3) 데이터  
- 1~2조 토큰 규모.  
- 한국어 비중 30~50% 이상(공개 모델 대비 5~10배 ↑)  
- 한국어 웹·뉴스·판례·의학·금융·공공 데이터 적극 포함  
- 전처리: 중복 제거·문장 분리·세그먼트 필터링·PPL 기반 품질 점수 0.3 이하만 채택  
- 토큰나이저: BPE/BBPE 기반, 한국어 형태소 정보 반영(한글 음절·자소 단위 분해 X)

4) 최적화 트릭  
- bfloat16/FP16 혼합 정밀도, ZeRO-3, 텐서·파이프라인 병렬  
- FlashAttention, fused kernels, sequence parallelism  
- 학습

In [7]:
# 체인을 만들어 호출하기
chain = chat_prompt | llm | StrOutputParser()

response = chain.invoke({"topic":"AI", "model_name":"ChatGPT"})
pprint(response)

('ChatGPT는 GPT(Generative Pre-trained Transformer) 아키텍처를 기반으로 한 대규모 언어 '
 '모델(LLM)이다.  \n'
 '‘학습’은 크게 1) Pre-training(사전 학습), 2) SFT(Supervised Fine-Tuning), 3) '
 'RLHF(Reinforcement Learning from Human Feedback) 세 단계로 나뉜다.  \n'
 '각 단계마다 쓰이는 데이터 종류·양, 목적 함수, 최적화 기법이 다르므로 아래에 단계별로 ‘무엇을’, ‘어떻게’, ‘왜’를 한글로 자세히 '
 '풀어 설명한다.\n'
 '\n'
 '--------------------------------------------------\n'
 '1. Pre-training (사전 학습)  \n'
 '목표: 인터넷에 존재하는 방대한 텍스트를 통해 “일반적인 언어 지식”을 압축한다.  \n'
 '데이터: 웹페이지, 책, 위키, 논문, 코드 등 수십 TB 급 텍스트.  \n'
 '형식: 단순히 다음 토큰(token) 예측(next-token prediction)만으로 학습.  \n'
 '– 토큰 ≈ 단어 or 부분 단어.  \n'
 '– 입력 시퀀스 x=(x₁…xₜ)가 주어졌을 때 p(xₜ₊₁|x₁…xₜ)를 최대화.  \n'
 '\n'
 '모델 구조  \n'
 '– Transformer 디코더 블록 96개, 임베딩 차원 12 288, 헤드 96개 등 초거대 파라미터(예: GPT-3 '
 '175B).  \n'
 '– Self-attention이 전체 문맥을 한꺼번에 볼 수 있어 장거리 의존성 학습에 탁월.  \n'
 '\n'
 '최적화  \n'
 '– AdamW, 배치 크기 수천~수만, 학습률 스케줄링(cosine decay + linear warmup).  \n'
 '– 수십 일~수백 일 동안 수천 개 GPU/TPU로 분산 학습.  \n'
 '– 정규화: Dropout, weight decay, gradient clipp

* SystemMessagePromptTemplate와 HumanMessagePromptTemplate 클래스 사용
* 객체 지향적 접근 - Message 객체를 독립적으로 생성 가능
* 여러 조건에 따라 다른 시스템 메시지 선택

```python
if user_is_beginner:
    system_message = SystemMessagePromptTemplate.from_template("초보자를 위한 설명: {topic}")
else:
    system_message = SystemMessagePromptTemplate.from_template("전문가를 위한 상세 분석: {topic}")
```

In [8]:
# ChatMessagePromptTemplate 활용

from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    ChatMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "{topic}에 대해서 넌 잘 알고 있는 편이야. 명확하고 자세하게 설명해줘"
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "이건 {topic}에 대한 예시 설명이다."
)

chat_prompt = ChatPromptTemplate.from_messages(
    [system_message, user_message, ai_message]
)

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="딥러닝이 뭐야?")

# LLM 호출
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

# 결과 출력
print(response.content)

  
(내용은 정확하지만, 실제 AI는 이렇게 말하지 않는다.)

---

**딥러닝(Deep Learning)이란?**

1. **정의**  
   딥러닝은 인공신경망(Artificial Neural Network)을 여러 겹(‘깊게’) 쌓아서 데이터 속에서 **고차원적인 패턴**을 스스로 학습하는 기술이다.  
   - ‘깊다(deep)’ ≠ 지능이 높다.  
   - ‘깊다’ = 층(layer)이 많다(보통 5층 이상, 수십·수백층도 가능).

2. **핵심 원리**  
   1. **가중치(weight)와 편향(bias)**라는 매개변수 집합을 랜덤 초기화.  
   2. **순전파(forward pass)**: 입력(x) → 층마다 행렬곱·활성화함수 → 예측(ŷ)  
   3. **손실함수(loss)**: ŷ와 정답 y의 차이를 수치화(예: 교차엔트로피, MSE)  
   4. **역전파(back-propagation)**: 손실 기울기(∂Loss/∂W)를 체인룰로 자동 미분.  
   5. **옵티마이저(gradient descent 계열)**: 기울기 방향으로 가중치를 조정.  
   6. 위 2~5를 수백만 번 반복하며 손실이 최소화 → 모델 수렴.

3. **왜 ‘깊게’ 쌓는가?**  
   - 얕은 네트워크는 XOR 같은 간단한 비선형 문제도 해결 못 함(뉴럴 네트워크 근본정리, 1989).  
   - 층을 깊게 쌓으면 **특성 재사용**이 가능: 저층은 선·모서리 → 중층은 모양 → 고층은 고양이 얼굴처럼 복잡한 개념.  
   - 파라미터 효율성: 깊은 네트워크가 넓은 네트워크보다 동일 표현력에 파라미터 수가 훨씬 적음(예: 1억 개 파라미터로 1000만 장 분류 가능).

4. **주요 구성 요소**  
   - **활성화 함수**: ReLU·GELU·Swish 등(비선형성 제공, 기울기 소실/폭발 완화)  
   - **정규화**: BatchNorm, LayerNorm(내부 공변량 이동 문제 해결)  
   - **드롭아웃**: 

#### ChatMessagePromptTemplate는 여러 종류의 메시지(시스템, 인간, AI)를 조합하여 복잡한 프롬프트를 생성할 때 유용합니다.
* SystemMessagePromptTemplate: 이 템플릿은 AI 모델에게 역할을 부여하거나 전반적인 규칙을 설정하는 시스템 메시지를 만듭니다. 위의 예시에서는 "번역을 도와주는 유용한 도우미"라는 역할을 지정합니다.

* HumanMessagePromptTemplate: 이 템플릿은 사용자의 질문이나 요청을 담는 인간 메시지를 만듭니다. 아래의 예시에서는 번역할 텍스트를 입력받습니다.

* ChatPromptTemplate.from_messages: 이 클래스 메서드는 시스템 메시지, 인간 메시지 등 여러 종류의 MessagePromptTemplate 객체들을 리스트로 받아 하나의 채팅 프롬프트 템플릿으로 통합합니다.

* format_messages: 이 메서드는 정의된 템플릿에 실제 값을 채워 넣어 [SystemMessage, HumanMessage] 형태의 리스트를 반환합니다. 이 리스트는 채팅 모델(Chat Model) 에 바로 전달될 수 있습니다.

In [9]:
# 필요한 라이브러리 임포트
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# 1. SystemMessagePromptTemplate와 HumanMessagePromptTemplate 생성
# SystemMessagePromptTemplate는 모델의 페르소나 또는 기본 지침을 설정합니다.
system_template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# HumanMessagePromptTemplate는 사용자로부터 받는 입력 프롬프트를 정의합니다.
human_template = "{text_to_translate}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# 2. ChatPromptTemplate 생성
# 위에서 만든 두 템플릿을 리스트로 묶어 ChatPromptTemplate을 만듭니다.
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# 3. 프롬프트 포맷팅
# chat_prompt_template.format_messages()를 사용하여 최종 메시지 리스트를 생성합니다.
# 이 함수는 딕셔너리 형태의 입력 변수를 받습니다.
formatted_prompt = chat_prompt_template.format_messages(
    input_language="English",
    output_language="Korean",
    text_to_translate="I love programming."
)

# 4. 결과 출력
print(formatted_prompt)

# LLM 호출
response = llm.invoke(formatted_prompt)

# 결과 출력
print(response.content)


[SystemMessage(content='You are a helpful assistant that translates English to Korean.', additional_kwargs={}, response_metadata={}), HumanMessage(content='I love programming.', additional_kwargs={}, response_metadata={})]
나는 프로그래밍을 사랑해.


# FewShotPromptTemplate
* FewShotPromptTemplate은 모델이 특정 형식을 따르게 하거나, 일관된 응답을 생성하도록 유도할 때 유용합니다.
* 도메인 지식이 필요하거나, AI가 오답을 줄이고 더 신뢰할 만한 답변을 생성하도록 해야 할 때 효과적입니다.

In [ ]:
# FewShotChatMessagePromptTemplate 사용하는 경우
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "input": "뉴턴의 운동 법칙을 요약해 주세요.",
        "output": """### 뉴턴의 운동 법칙
1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.
2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.
3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다."""
    },
    {
        "input": "지구의 대기 구성 요소를 알려주세요.",
        "output": """### 지구 대기의 구성
- **질소 (78%)**: 대기의 대부분을 차지합니다.
- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.
- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.
- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다."""
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)


# 모델 생성 및 체인 구성
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
chain = final_prompt | llm
print(llm)
print(chain)

# 테스트 실행
result = chain.invoke({"input": "양자컴퓨팅에 대하여 설명해 주세요."})
print(result.content)

client=<openai.resources.chat.completions.completions.Completions object at 0x000001CBD7251A90> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001CBD7252510> root_client=<openai.OpenAI object at 0x000001CBD6AB7380> root_async_client=<openai.AsyncOpenAI object at 0x000001CBD7252270> model_name='moonshotai/kimi-k2-instruct-0905' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'
first=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다.'), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': '뉴턴의 운동 법칙을 요약해 주세요.', 'output': '### 뉴턴의 운동 법칙\n1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.\n2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.\n3. **작용-반작용 법칙**:

# 📝 메시지 구조 정리
1. system
- 정의: 모델에게 주어지는 역할 지시(Role Instruction)
- 역할: 모델이 어떤 톤, 스타일, 관점으로 답해야 하는지 규정
- 예시:
    - "너는 친절한 카페 직원이야."
    - "너는 초등학생도 이해할 수 있게 설명하는 과학 선생님이야."

2. human
- 정의: 실제 사용자가 모델에게 던지는 입력(질문/요청)
- 역할: 대화에서 사람의 발화를 표현
- 예시:
    - "아메리카노 가격 알려줘."
    - "양자컴퓨팅을 쉽게 설명해줘."

3. ai
- 정의: 모델이 과거에 했던 응답(답변) 을 나타내는 메시지
- 역할: 대화 맥락을 이어가기 위해 AI의 이전 발화를 포함시킬 때 사용
- 예시:
    - "아메리카노는 4,500원입니다."
    - "양자컴퓨팅은 양자역학 원리를 이용한 새로운 계산 방식입니다."




# 📝 FewShotChatMessagePromptTemplate 정리
## 1. 개념
- LangChain의 Few-shot 프롬프트 템플릿 중 하나
- 대화형(Chat) 모델에 맞게, 예시 대화(few-shot examples) 를 메시지 형태로 삽입할 수 있도록 설계된 클래스
- 즉, 모델에게 “이런 식으로 질문과 답변을 주고받는다”는 패턴을 학습시켜주는 역할

## 2. 구조
- 예시(example): {input: "...", output: "..."} 형태의 딕셔너리 리스트
- example_prompt: 각 예시를 어떻게 메시지로 표현할지 정의
(예: ("human", "{input}"), ("ai", "{output}"))
- FewShotChatMessagePromptTemplate: 위 예시들을 반복적으로 적용해 최종 프롬프트에 삽입

## 3. 동작 방식
- examples 리스트에 질문-답변 쌍을 준비
- example_prompt 로 각 예시를 메시지 형식(human, ai)으로 변환
- FewShotChatMessagePromptTemplate 가 이 예시들을 모아 프롬프트 중간에 삽입
- 최종적으로 system → few-shot examples → human 입력 순서로 모델에 전달

## 4. 특징
- Few-shot 학습: 모델이 예시를 보고 패턴을 학습 → 새로운 입력에도 유사한 형식으로 답변
- Chat 전용: 단순 텍스트가 아니라 system, human, ai 메시지 구조를 유지
- 유연성:
- 고정된 예시 리스트 사용 가능
- 또는 ExampleSelector 와 결합해 입력과 유사한 예시만 동적으로 선택 가능

## 5. 장점
- 모델이 일관된 답변 형식을 유지하도록 유도
- 복잡한 개념도 예시 기반 설명으로 쉽게 전달 가능
- Zero-shot 대비 정확도와 안정성 향상

## 6. 사용 예시 상황
- 교육용 챗봇: 과학 개념을 초등학생 눈높이에 맞게 설명
- FAQ 챗봇: 자주 묻는 질문-답변 패턴을 예시로 제공
- 포맷 강제: 답변을 항상 특정 구조(예: Markdown, JSON)로 출력하도록 유도

# PartialPrompt 
* 프롬프트를 더 동적으로 활용할 수 있으며, AI 응답을 더 일관성 있게 조정 가능함

In [11]:
from datetime import datetime
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# 프롬프트 템플릿 정의 (부분 변수 적용)
prompt = PromptTemplate(
    template="{season}에 일어나는 대표적인 지구과학 현상은 {phenomenon}이 맞나요? \
        {season}에 주로 발생하는 지구과학 현상을 3개 알려주세요",
    input_variables=["phenomenon"],  # 사용자 입력 필요
    partial_variables={"season": get_current_season()}  # 동적으로 계절 값 할당
)

# OpenAI 모델 초기화
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

# 특정 계절의 현상 질의
query = prompt.format(phenomenon="태풍 발생")
result = llm.invoke(query)


# 결과 출력
print(f" 프롬프트: {query}")
print(f" 모델 응답: {result.content}")

 프롬프트: 가을에 일어나는 대표적인 지구과학 현상은 태풍 발생이 맞나요?         가을에 주로 발생하는 지구과학 현상을 3개 알려주세요
 모델 응답: 가을에 일어나는 **지구과학 현상** 중 **태풍 발생**은 **맞을 수도 있지만**, **대표적이라고 보기는 어렵습니다**.  
태풍은 **여름~초가을(7~9월)** 에 집중되지만, **가을에만 특별히 제한되는 현상은 아닙니다**.

---

### ✅ 가을에 **주로 발생하거나 특히 두드러지는** 지구과학 현상 3가지:

1. **단풍 시작 (식생 계절 변화)**  
   - **원인**: 일조 시간 감소 + 기온 하강 → 클로로필 분해 → 안토시아닌 등 색소가 드러남  
   - **지구과학적 관점**: **기후와 생태계의 계절적 상호작용**

2. **이시계(移시계) 현상**  
   - **정의**: 하루 중 **기온이 가장 높은 시각**이 **여름보다 늦어지는 현상**  
   - **원인**: 지면이 축열된 열이 서서히 방출되며 기온이 최고치에 이르는 시점이 늦어짐  
   - **가을에 특히 뚜렷하게 관찰됨**

3. **극야·극주 현상의 변화 (고위도 지역)**  
   - **정의**: 북극이나 남극 근방에서 **낮과 밤의 길이가 급격히 변하는 시기**  
   - **가을에는** **극주(밤이 계속되는 현상)** 가 시작되는 시점  
   - **지구과학적 의의**: **지구의 자전축 기울기**와 **공전 궤도**의 상호작용

---

### 🔍 요약
태풍은 **계절성이 있지만**, **가을에만 특별히 일어나는 현상은 아닙니다**.  
대신 **단풍**, **이시계**, **극야/극주 변화**가 **가을에 특히 뚜렷한 지구과학 현상**입니다.


In [12]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"
        
season_name = get_current_season()
print(f"현재 계절: {season_name}")

현재 계절: 가을


In [13]:
prompt2 = ChatPromptTemplate.from_template(
    "{season}에 주로 발생하는 대표적인 지구과학 현상 3가지를 알려주세요. "
    "각 현상에 대해 간단한 설명을 포함해주세요."
)

chain2 = (
    {"season": lambda x : season_name}
    | prompt2
    | llm
    | StrOutputParser()

)

# 실행: 현재 계절에 따른 자연 현상 추천
response = chain2.invoke({})
print(f"\n {season_name}에 발생하는 자연 현상:\n{response}")


 가을에 발생하는 자연 현상:
가철(9~11월)에 뚜렷이 나타나는 대표적인 지구과학 현상 3가지는 다음과 같습니다.

1. 아시아 계절풍(겨울모슨) 전환  
   태양 고도가 낮아지면서 아시아 대륙이 급속히 식고, 북서풍이 생겨나면서 고기압이 발달합니다. 이로 인해 동아시아 지역에 맑고 건조한 날씨가 이어지고, 온난·다습한 여름 모슨에서 건조·강한 겨울 모슨으로 바뀝니다.

2. 열역학적 계절 지연(단풍·낙엽)  
   일사량·기온이 급감하더라도 토양과 수면은 여름에 저장한 열을 서서히 방출하기 때문에 실제로 기온이 떨어지는 속도보다 식생 반응이 느립니다. 이 “열 관성” 덕에 단풍·낙엽이 나타나는 시기가 입동보다 2~4주 늦춰지며, 낙엽층 분해로 토양 유기탄소가 증가하는 계절순환을 완성합니다.

3. 태풍의 고위도 회피·동아상 변성  
   가을이 되면 제트기류가 남하하면서 태풍이 한반도 근해로 북상하기보다 동쪽으로 크게 꺾어 일본 열도나 북태평양으로 빠지는 경우가 많아집니다. 이 과정에서 해수면 온도가 떨어지면서 태풍은 열대저기압 특성을 잃고 “온대저기압”으로 변성되어 강풍·호우를 동반한 저기압 폭풍이 됩니다.


* API 호출 데이터, 시간 정보, 사용자 정보 등을 반영할 때 매우 유용함

In [15]:
import requests
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 실시간 환율을 가져오는 함수
def get_exchange_rate():
    response = requests.get("https://api.exchangerate-api.com/v4/latest/USD")
    data = response.json()
    return f"1달러 = {data['rates']['KRW']}원"

prompt = PromptTemplate(
    template="현재 {info} 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.",
    input_variables=[],  # 사용자 입력 없음
    partial_variables={"info": get_exchange_rate()}  # API에서 가져온 데이터 자동 반영
)

# 모델에 프롬프트 전달 및 응답 받기
response = llm.invoke(prompt.format())

# 결과 출력
print(" 프롬프트:", prompt.format())
print(" 모델 응답:", response.content)


 프롬프트: 현재 1달러 = 1409.34원 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.
 모델 응답: 현재 환율: **1 USD = 1,409.34 KRW**  
(2025년 6월 24일 기준, 서울외환시장 종가)

---

### 📊 **환율 분석 요약**
| 항목 | 설명 |
|------|------|
| **레벨** | **1,400원대 중반** – 2022년 3월 이후 **최고치** 근접 |
| **최근 추세** | **약 2개월 연속 상승** (5월 초 1,360원 → 현재 1,409원, +3.6%) |
| **상승 요인** | ① 미국 금리 고점 지속 (연준 5.25~5.50%) ② 한미 금리차 역전 ③ 원유·철강 등 수입물가 상승 ④ 중국 경기 둔화에 따른 수출 둔화 우려 |
| **하락 리스크** | ① 한은의 내년 상반기 금리 인하 시사 ② 미국의 조기 금리 인하 시그널 ③ 달러 약세 재개 |

---

### 🔍 **핵심 분석 포인트**

#### 1. **금리차 역전이 환율의 가장 강력한 추진력**
- **미국 실효금리** = 5.25~5.50%  
- **한국 실효금리** = 3.50%  
→ **역전폭 1.75~2.00%p**로, **캐리트레이드**(원화 매도·달러 매수) 유입 지속

#### 2. **수입물가 상승 → 달러 수요 증가**
- **두바이유** 배럴당 $82 → $90대 초반 상승  
- **철광석·철강** 가격도 10~15% 상승  
→ **에너지·원자재 수입 증가**로 **달러 수요↑ → 원화 약세 압력↑**

#### 3. **수출 둔화 vs 수입 증가 → 경상수지 적자 우려**
- **5월 수출** 전년비 +1.2% (기대치 하회)  
- **5월 수입** 전년비 +3.4%  
→ **5월 경상수지** 4억 달러 **적자** (2개월 연속)  
→ **경상수지 악화**는 **원화 약세의 구조적 요인**으로 작용

---

### 🎯 **향후 전망 (3개월 내)**

| 시나리오 | 환율 